In [1]:
pip install torch torchvision matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cpu')

In [5]:
tf=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])
])

In [6]:
train_dl=DataLoader(
    datasets.ImageFolder('data/Training',tf),
    batch_size=32, shuffle=True, num_workers=4, pin_memory=True
)


test_dl=DataLoader(
    datasets.ImageFolder('data/Testing',tf),
    batch_size=32, shuffle=False, num_workers=4, pin_memory=True
)

In [7]:
train_dl

In [8]:
model=nn.Sequential(
    nn.Conv2d(3, 32,3,1,1),nn.ReLU(),nn.MaxPool2d(2),
    nn.Conv2d(32, 64,3,1,1),nn.ReLU(),nn.MaxPool2d(2),
    nn.Conv2d(64, 128,3,1,1),nn.ReLU(),nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(128*16*16,256),nn.ReLU(),nn.Dropout(0.5),
    nn.Linear(256,4)
).to(device)

In [9]:
opt=optim.AdamW(model.parameters())
loss_fn=nn.CrossEntropyLoss()

In [10]:
model.train()

for epoch in range(18):
    running_loss=0
    
    for x,y in train_dl:
        opt.zero_grad()
        
        loss=loss_fn(model(x.to(device)),y.to(device))
        loss.backward()
        
        running_loss+=loss
        opt.step()
    print(f'Epoch {epoch+1}: Loss was {running_loss}')

Epoch 1: Loss was 117.19095611572266
Epoch 2: Loss was 67.69306182861328
Epoch 3: Loss was 44.853416442871094
Epoch 4: Loss was 32.49208450317383
Epoch 5: Loss was 26.159465789794922
Epoch 6: Loss was 19.285266876220703
Epoch 7: Loss was 14.52657699584961
Epoch 8: Loss was 14.338025093078613
Epoch 9: Loss was 8.781617164611816
Epoch 10: Loss was 9.031197547912598
Epoch 11: Loss was 6.418173789978027
Epoch 12: Loss was 6.080014705657959
Epoch 13: Loss was 5.472994327545166
Epoch 14: Loss was 5.6801934242248535
Epoch 15: Loss was 5.118819236755371
Epoch 16: Loss was 5.1695237159729
Epoch 17: Loss was 4.958931922912598
Epoch 18: Loss was 3.6825435161590576


In [11]:
model.eval()
test_loss, correct=0.0,0

with torch.no_grad():
    for x, y in test_dl:
        x,y=x.to(device),y.to(device)
        
        logits=model(x)
        test_loss+=loss_fn(logits,y).item()*y.size(0)
        
        preds=logits.argmax(dim=1)
        correct+=(preds==y).sum().item()
        
test_loss/=len(test_dl.dataset)
accuracy=100.0*correct/len(test_dl.dataset)

print('Test Loss:', test_loss, 'Test Accuracy', accuracy,)

Test Loss: 0.11349023483295625 Test Accuracy 97.17772692601068


In [12]:
import torch

# Save the entire model
torch.save(model, 'brain_tumor_model.pth')

# Or save just the state dictionary (recommended)
torch.save(model.state_dict(), 'brain_tumor_model_weights.pth')

# Also save the class names for later use
import pickle
class_names = train_dl.dataset.classes
with open('class_names.pkl', 'wb') as f:
    pickle.dump(class_names, f)

print("Model saved successfully!")


Model saved successfully!


In [ ]:
import random
import matplotlib.pyplot as plt
from torchvision.transforms.functional import to_pil_image


model.eval()

idx= random.randrange(len(test_dl.dataset))
img,label=test_dl.dataset[idx]

unnorm=img*0.5+0.5
plt.imshow(to_pil_image(unnorm))
plt.axis('off')
plt.title('Sample from test set')
plt.show()


with torch.no_grad():
    logits=model(img.unsqueeze(0).to(device))
    pred=logits.argmax(1).item()
    
class_names=test_dl.dataset.classes
print(f'Predicted class:{class_names[pred]}')
print(f'Ground-truth: {class_names[label]}')